In [0]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,Model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Input
from sklearn.model_selection import train_test_split
import re

In [0]:
from google.colab import files
upload=files.upload()

Saving dataset_ingles_hom_shakespeare.csv to dataset_ingles_hom_shakespeare (1).csv


In [0]:
datos=pd.read_csv('dataset_ingles_hom_shakespeare.csv')

In [0]:
datos.head()

,Unnamed: 0,fragmentos,autor
0,0,many a brave soul did it send hurrying down t...,Homero-t-i
1,1,and which of the gods was it that set them on ...,Homero-t-i
2,2,now chryses had e to the ships of the achaean...,Homero-t-i
3,3,"‘sons of atreus,’ he cried, ‘and all other ach...",Homero-t-i
4,4,’on this the rest of the achaeans with one voi...,Homero-t-i


In [0]:
datos = datos[['fragmentos','autor']]

In [0]:
# def quit(x):
#     x=str(x)[8:]
#     return x

In [0]:
datos.head()

,fragmentos,autor
0,many a brave soul did it send hurrying down t...,Homero-t-i
1,and which of the gods was it that set them on ...,Homero-t-i
2,now chryses had e to the ships of the achaean...,Homero-t-i
3,"‘sons of atreus,’ he cried, ‘and all other ach...",Homero-t-i
4,’on this the rest of the achaeans with one voi...,Homero-t-i


In [0]:
datos['fragmentos'] = datos['fragmentos'].apply(lambda x: str(x).lower())
datos['fragmentos'] = datos['fragmentos'].apply((lambda x: re.sub('[^a-zA-z0-9Ñ-ñ\s]','',x)))
# datos['ejemplos'] = datos['ejemplos'].apply(lambda x: quit(x))

In [0]:
datos['fragmentos'][4]

'on this the rest of the achaeans with one voice were for respecting the priest and taking the ransom that he offered free ebooks at planet ebook'

In [0]:
datos['fragmentos'].shape

(13027,)

In [0]:
kk=set()
for i in datos['fragmentos']:
  for k in i.split(" "):
    kk.add(k)
len(kk)

14541

In [0]:
maxim = len(kk)

tokenizer = Tokenizer(num_words=maxim, split=' ')
tokenizer.fit_on_texts(datos['fragmentos'].values)
X = tokenizer.texts_to_sequences(datos['fragmentos'].values)
X = pad_sequences(X)

In [0]:
y=[x for x in datos['autor']]

labelencoder_y=LabelEncoder()
labelencoder_y.fit(np.asarray(y))
yi=labelencoder_y.transform(np.asarray(y))

In [0]:
clases={}
for i in range(len(yi)):
    clases[yi[i]]=y[i]
print(len(yi))

13027


In [0]:
clases

{0: 'Homero-t-i',
 1: 'shakespeare-hmlet',
 2: 'shakespeare-merch-v',
 3: 'shakespeare-rom-jul',
 4: 'shakespeare-sonnet'}

In [0]:
cx=yi.reshape(len(X),1).astype("float32")
onehotencoder=OneHotEncoder(categorical_features=[0])
Y=onehotencoder.fit_transform(cx).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [0]:
yi.shape

(13027,)

In [0]:

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 55)

In [0]:
entrada=Input((None,))
embedding=Embedding(maxim, 250,input_length = X.shape[1])(entrada)
dropout=SpatialDropout1D(0.4)(embedding)

lstm=LSTM(200, dropout=0.2, recurrent_dropout=0.2)(dropout)
out=Dense(5,activation='softmax')
salida=out(lstm)
model=Model(entrada,salida)
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

W0722 21:03:19.803007 140058434344832 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0722 21:03:19.858853 140058434344832 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0722 21:03:19.870442 140058434344832 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0722 21:03:19.897445 140058434344832 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0722 21:03:19.909055 

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 217, 250)          3635250   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 217, 250)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               360800    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1005      
Total params: 3,997,055
Trainable params: 3,997,055
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_train, Y_train, epochs = 5, batch_size=100)

W0722 21:03:33.436013 140058434344832 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
10421/10421 [==============================] - 243s 23ms/step - loss: 1.0170 - acc: 0.5905
Epoch 2/5
10421/10421 [==============================] - 239s 23ms/step - loss: 0.5344 - acc: 0.7924
Epoch 3/5
10421/10421 [==============================] - 240s 23ms/step - loss: 0.3490 - acc: 0.8692
Epoch 4/5
10421/10421 [==============================] - 240s 23ms/step - loss: 0.2496 - acc: 0.9067
Epoch 5/5
10421/10421 [==============================] - 238s 23ms/step - loss: 0.1919 - acc: 0.9277


In [0]:
# model.save_weights('pesos_nlp_lstm_embeddings.h5')
# model.load_weights('pesos_nlp_lstm_embeddings.h5')

In [0]:
score,acc = model.evaluate(X_test, Y_test, batch_size =100)

2606/2606 [==============================] - 14s 5ms/step


In [0]:
print("score={},acc={}".format(score,acc))

score=0.5196282822332836,acc=0.8250191864012208


In [0]:
def predecir(x):

    word=["{}".format(x)]
    word=tokenizer.texts_to_sequences(word)
    word=pad_sequences(word, maxlen=106, dtype='int32', value=0)
    predict=model.predict(word)[0][:]
    predict_clase=np.where(predict == max(predict))[0]
    
    return clases[int(predict_clase[0])]

In [0]:
# word=["ford falcon 4puertas mod 82"]
# word=tokenizer.texts_to_sequences(word)
# word=pad_sequences(word, maxlen=106, dtype='int32', value=0)
# clases[int(model.predict_classes(word))]
#esto solo sirve para model Sequential()

# las predicciones se hacen a partir de ver el dataset y utilizar vocabulario del mismo o no inventando los anuncios

In [0]:
predecir('zona centro 4amb cocina-comedor,1hab, living y parque u$s 175000')

'Casas(388)'

In [0]:
predecir('señora 45años 10 años experiencia cuidado de mayores')

'Cuidado de Personas(463)'

In [0]:
predecir('palermo 3amb cocina,comedor 2hab u$s 120300 segundo piso')

'Departamentos(3014)'